In [1]:
# 线性回归
主要内容包括：

1. 线性回归的基本要素
2. 线性回归模型从零开始的实现
3. 线性回归模型使用pytorch的简洁实现

SyntaxError: invalid character in identifier (<ipython-input-1-af033d0f4323>, line 2)

## 线性回归的基本要素

### 模型
为了简单起见，这里我们假设价格只取决于房屋状况的两个因素，即面积（平方米）和房龄（年）。接下来我们希望探索价格与这两个因素的具体关系。线性回归假设输出与各个输入之间是线性关系:


$$
\mathrm{price} = w_{\mathrm{area}} \cdot \mathrm{area} + w_{\mathrm{age}} \cdot \mathrm{age} + b
$$



### 数据集
我们通常收集一系列的真实数据，例如多栋房屋的真实售出价格和它们对应的面积和房龄。我们希望在这个数据上面寻找模型参数来使模型的预测价格与真实价格的误差最小。在机器学习术语里，该数据集被称为训练数据集（training data set）或训练集（training set），一栋房屋被称为一个样本（sample），其真实售出价格叫作标签（label），用来预测标签的两个因素叫作特征（feature）。特征用来表征样本的特点。
### 损失函数
在模型训练中，我们需要衡量价格预测值与真实值之间的误差。通常我们会选取一个非负数作为误差，且数值越小表示误差越小。一个常用的选择是平方函数。 它在评估索引为 $i$ 的样本误差的表达式为


$$
l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2,
$$



$$
L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2.
$$


### 优化函数 - 随机梯度下降
当模型和损失函数形式较为简单时，上面的误差最小化问题的解可以直接用公式表达出来。这类解叫作解析解（analytical solution）。本节使用的线性回归和平方误差刚好属于这个范畴。然而，大多数深度学习模型并没有解析解，只能通过优化算法有限次迭代模型参数来尽可能降低损失函数的值。这类解叫作数值解（numerical solution）。

在求数值解的优化算法中，小批量随机梯度下降（mini-batch stochastic gradient descent）在深度学习中被广泛使用。它的算法很简单：先选取一组模型参数的初始值，如随机选取；接下来对参数进行多次迭代，使每次迭代都可能降低损失函数的值。在每次迭代中，先随机均匀采样一个由固定数目训练数据样本所组成的小批量（mini-batch）$\mathcal{B}$，然后求小批量中数据样本的平均损失有关模型参数的导数（梯度），最后用此结果与预先设定的一个正数的乘积作为模型参数在本次迭代的减小量。   

$$
(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b)
$$
  
学习率: $\eta$代表在每次优化中，能够学习的步长的大小    
批量大小: $\mathcal{B}$是小批量计算中的批量大小batch size   

总结一下，优化函数的有以下两个步骤：

- (i)初始化模型参数，一般来说使用随机初始化；
- (ii)我们在数据上迭代多次，通过在负梯度方向移动参数来更新每个参数。

## 矢量计算
在模型训练或预测时，我们常常会同时处理多个数据样本并用到矢量计算。在介绍线性回归的矢量计算表达式之前，让我们先考虑对两个向量相加的两种方法。


1. 向量相加的一种方法是，将这两个向量按元素逐一做标量加法。
2. 向量相加的另一种方法是，将这两个向量直接做矢量加法。

In [2]:
import torch
import time

# init variable a, b as 1000 dimension vector
n = 1000
a = torch.ones(n)
b = torch.ones(n)


In [5]:
a.size()

torch.Size([1000])

In [9]:
a_1=a.view(a.size(0),-1)
print(a_1.size())

torch.Size([1000, 1])


In [4]:
# define a timer class to record time
class Timer(object):
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        # start the timer
        self.start_time = time.time()

    def stop(self):
        # stop the timer and record time into a list
        self.times.append(time.time() - self.start_time)
        return self.times[-1]

    def avg(self):
        # calculate the average and return
        return sum(self.times)/len(self.times)

    def sum(self):
        # return the sum of recorded time
        return sum(self.times)

现在我们可以来测试了。首先将两个向量使用for循环按元素逐一做标量加法。

In [5]:
timer = Timer()
c = torch.zeros(n)
for i in range(n):
    c[i] = a[i] + b[i]
'%.5f sec' % timer.stop()

'0.01142 sec'

另外是使用torch来将两个向量直接做矢量加法：

In [6]:
timer.start()
d = a + b
'%.5f sec' % timer.stop()

'0.00022 sec'

结果很明显,后者比前者运算速度更快。因此，我们应该尽可能采用矢量计算，以提升计算效率。

## 线性回归模型从零开始的实现



In [2]:
# import packages and modules
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

print(torch.__version__)

1.3.0


### 生成数据集
使用线性模型来生成数据集，生成一个1000个样本的数据集，下面是用来生成数据的线性关系：

$$
\mathrm{price} = w_{\mathrm{area}} \cdot \mathrm{area} + w_{\mathrm{age}} \cdot \mathrm{age} + b
$$



In [4]:
# set input feature number 
num_inputs = 2
# set example number
num_examples = 1000

# set true weight and bias in order to generate corresponded label
true_w = [2, -3.4]
true_b = 4.2

features = torch.randn(num_examples, num_inputs,
                      dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)#  list, tuple, array, scalar 转化为tensor

In [5]:
tensor_1=torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)             #   
                       

In [6]:
print(tensor_1.size())

torch.Size([1000])


In [7]:
features.size()

torch.Size([1000, 2])

In [8]:
labels.size()

torch.Size([1000])

In [9]:
labels.size()

torch.Size([1000])

In [10]:
features[:,0].size()

torch.Size([1000])

In [11]:
true_w[0]

2

### 使用图像来展示生成的数据

In [13]:
plt.scatter(features[:, 1].numpy(), labels.numpy(), 1);  # tensor.numpy()表示将tensor 转化为Numpy

<Figure size 432x288 with 1 Axes>

In [12]:
print(features)   #   .size(0) 表示行数  .size(1)表示列数
print(features.size(0))
print(features.size(1))

tensor([[ 0.8877,  0.2456],
        [-1.0391,  0.7241],
        [-0.5199, -0.0042],
        ...,
        [ 0.5400, -0.3943],
        [-0.7943,  0.3329],
        [-0.3577, -0.6104]])
1000
2


### 读取数据集

In [13]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # 打乱indices 这个list 里面的数字的顺序
    for i in range(0, num_examples, batch_size):  #取0,10,20,,,990
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # the last time may be not enough for a whole batch
        yield  features.index_select(0, j), labels.index_select(0, j) 
        #  返回一个迭代器包含分别features和lables里指定的行构成的tensor ,行索引在j 中

In [16]:
#  弄懂data_iter函数过程
batch_size_1=10
num_examples_1=1000
indices_1=list(range(num_examples_1))
random.shuffle(indices_1)
for i_1 in range(0, 1000, 10):
    j_1 = torch.LongTensor(indices_1[i_1: min(i_1 + batch_size_1, num_examples_1)]) # the last time may be not enough for a whole batch
    if i_1==0:
        print(j_1)
        print(features.index_select(0, j_1))
        print( labels.index_select(0, j_1))
    # yield  features.index_select(0, j), labels.index_select(0, j)

tensor([833, 393, 418,  61, 327, 155, 165, 890,  39, 168])
tensor([[-2.0612, -0.9871],
        [ 1.8983,  1.4066],
        [-0.0882, -1.2238],
        [ 1.7690,  0.7598],
        [-0.2678, -0.3951],
        [ 0.2469,  0.2984],
        [ 1.6884,  0.9489],
        [ 1.5071, -0.9317],
        [-1.3891, -0.5439],
        [ 0.3203,  0.7078]])
tensor([ 3.4173,  3.2047,  8.1748,  5.1670,  5.0062,  3.6648,  4.3612, 10.3912,
         3.2692,  2.4205])


In [17]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-1.0281, -0.3254],
        [ 0.1331, -1.0631],
        [-3.2566, -0.1795],
        [ 1.3423,  1.1218],
        [ 0.0078, -0.0416],
        [-2.8044,  0.3354],
        [ 1.2262, -0.9178],
        [ 0.3653,  1.1076],
        [ 0.2707, -0.3150],
        [ 0.6833, -0.9698]]) 
 tensor([ 3.2448,  8.0680, -1.6894,  3.0663,  4.3628, -2.5552,  9.7673,  1.1754,
         5.8162,  8.8429])


In [28]:
X.shape

torch.Size([10, 2])

In [27]:
y.shape

torch.Size([10])

### 初始化模型参数

In [18]:
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)

w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], requires_grad=True)

In [26]:
np.random.normal(0, 0.01, (num_inputs,1))

array([[0.00599015],
       [0.01168193]])

In [27]:
print(b)

tensor([0.], requires_grad=True)


### 定义模型
定义用来训练参数的训练模型：

$$
\mathrm{price} = w_{\mathrm{area}} \cdot \mathrm{area} + w_{\mathrm{age}} \cdot \mathrm{age} + b
$$



In [28]:
def linreg(X, w, b):
    return torch.mm(X, w) + b

### 定义损失函数
我们使用的是均方误差损失函数：
$$
l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2,
$$


In [29]:
def squared_loss(y_hat, y): 
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

In [48]:
x_1=torch.arange(3)
x_1_0=x_1.view(3)
x_1_0_0=x_1.view(-1)  # 全部摊平得到一维数组，可以看成列向量
x_1_1=x_1.view(3,-1)   #  要得到二维数组 且行数是三行，第二个参数-1 是自适应
x_1_2=x_1_1.view(-1)
y_1=torch.arange(2,5).view(3,1)
z_1=torch.arange(3).view(1,3)
print(x_1)
print(x_1.view(y_1.shape))
print(x_1_0)
print(x_1_0_0)
print(x_1_1)
print(x_1_2)
print(y_1)
print(z_1)
print(x_1+y_1) #  相当于用x_1 的每一个元素加上y_1 的所有元素

tensor([0, 1, 2])
tensor([[0],
        [1],
        [2]])
tensor([0, 1, 2])
tensor([0, 1, 2])
tensor([[0],
        [1],
        [2]])
tensor([0, 1, 2])
tensor([[2],
        [3],
        [4]])
tensor([[0, 1, 2]])
tensor([[2, 3, 4],
        [3, 4, 5],
        [4, 5, 6]])


In [31]:
x_1.shape

torch.Size([3])

In [47]:
y_1.shape

torch.Size([3, 1])

In [33]:
z_1.shape

torch.Size([1, 3])

In [4]:
n_1=torch.arange(3,9)
n_2=n_1.view(3,-1)
n_3=n_1.view(n_2.size())
n_4=n_1.view(n_2.shape)
n_5=n_4.view(-1)
n_6=n_1.view(n_1.size(0),-1)
n_7=n_1.view(-1,n_1.size(0))
print(n_1)
print(n_2)
print(n_3)
print(n_4)
print(n_5)
print(n_6)
print(n_7)

tensor([3, 4, 5, 6, 7, 8])
tensor([[3, 4],
        [5, 6],
        [7, 8]])
tensor([[3, 4],
        [5, 6],
        [7, 8]])
tensor([[3, 4],
        [5, 6],
        [7, 8]])
tensor([3, 4, 5, 6, 7, 8])
tensor([[3],
        [4],
        [5],
        [6],
        [7],
        [8]])
tensor([[3, 4, 5, 6, 7, 8]])


### 定义优化函数
在这里优化函数使用的是小批量随机梯度下降：

$$
(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b)
$$
  

In [32]:
def sgd(params, lr, batch_size): 
    for param in params:
        param.data -= lr * param.grad / batch_size # ues .data to operate param without gradient track

### 训练
当数据集、模型、损失函数和优化函数定义完了之后就可来准备进行模型的训练了。


In [36]:
# super parameters init
lr = 0.03
num_epochs = 5

net = linreg
loss = squared_loss

# training
for epoch in range(num_epochs):  # training repeats num_epochs times
    # in each epoch, all the samples in dataset will be used once
    
    # X is the feature and y is the label of a batch sample
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  
        # calculate the gradient of batch sample loss 
        l.backward()  
        # using small batch random gradient descent to iter model parameters
        sgd([w, b], lr, batch_size)  
        # reset parameter gradient
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

epoch 1, loss 0.000050
epoch 2, loss 0.000050
epoch 3, loss 0.000050
epoch 4, loss 0.000050
epoch 5, loss 0.000051


In [30]:
w, true_w, b, true_b

(tensor([[ 0.0056],
         [-0.0103]], requires_grad=True),
 [2, -3.4],
 tensor([0.], requires_grad=True),
 4.2)

In [34]:
w.grad

tensor([[0.],
        [0.]])

## 线性回归模型使用pytorch的简洁实现


In [16]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

### 生成数据集
在这里生成数据集跟从零开始的实现中是完全一样的。

In [17]:
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

### 读取数据集

In [18]:
import torch.utils.data as Data

batch_size = 10

# combine featues and labels of dataset
dataset = Data.TensorDataset(features, labels)

# put dataset into DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,            # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # whether shuffle the data or not
    num_workers=2,              # read data in multithreading
)

In [19]:
for X, y in data_iter:
    print(X, '\n', y)
    break

### 定义模型

In [20]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()      # call father function to init 
        self.linear = nn.Linear(n_feature, 1)  # function prototype: `torch.nn.Linear(in_features, out_features, bias=True)`

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

In [21]:
# ways to init a multilayer network
# method one
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # other layers can be added here
    )

# method two
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# method three
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

### 初始化模型参数

In [22]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # or you can use `net[0].bias.data.fill_(0)` to modify it directly

In [23]:
for param in net.parameters():
    print(param)

### 定义损失函数

In [24]:
loss = nn.MSELoss()    # nn built-in squared loss function
                       # function prototype: `torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')`

### 定义优化函数

In [25]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)   # built-in random gradient descent function
print(optimizer)  # function prototype: `torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)`

### 训练

In [26]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # reset gradient, equal to net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

In [27]:
# result comparision
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

## 两种实现方式的比较
1. 从零开始的实现（推荐用来学习）

   能够更好的理解模型和神经网络底层的原理
   

2. 使用pytorch的简洁实现

   能够更加快速地完成模型的设计与实现
   